# 03 평가
## 머신러닝의 프로세스 = "데이터가공/변환", "모델 학습/예측", "평가"

## 머신러닝 모델의 예측 성능을 평가하는 성능 평가 지표(EvaluationMetric)
는 일반적으로 모델이 분류냐 회귀냐에 따라 여러종류로 나뉜다.

### 분류 : 실제 결과 데이터와 예측 결과데이터가 얼마나 정확하고 오류가 적게발생하는가에 기반하지만,
  단순히 정확도만 가지고 판단할경우 잘못된 평가결과에 빠질수있다. 
      
#### *분류( 결정 클래스값 종류의 유형에 따라 )
   1) 이진분류 : 긍정/부정과 같은 2개의 결괏값만을 가짐
   
   2) 멀티분류 : 여러개의 결정 클래스값을 가짐
   
##### 아래의 6개의 분류성능지표는 이진/멀티 분류 모두에 적용되는 지표.
 
 ##### 특히 이진분류에서 더욱 중요하게 강조되는 지표이다
 
      1) 정확도(Accuracy)
      
      2) 오차행렬(Confusion Matrix)
      
      3) 정밀도(Precision)
      
      4) 재현율(Recall)
      
      5) F1 스코어
      
      6) ROC AUC

 ### 회귀 : 대부분 실제값과 예측값의 오차평균값에 기반(5장에서 상세하게 설명)

## 01.정확도(Accuracy)

### 정확도 : 실제데이터에서 예측 데이터가 얼마나 같은지를 판단하는 지표

= ${예측결과가 동일한 데이터 건수 \over 전체 예측 데이터 건수}$

이는 직관적으로 모델예측성능을 나타내는 평가지표다.

하지만, 이진 분류의 경우 ML모델의 성능을 왜곡할수있음.

성능을 왜곡하는 예제를 살펴보자

타이타닉 예제에서 별다른 알고리즘 적용없이 무조건 성별이 여자인경우 생존으로 예측해도 비슷할수있다.

단지 성별조건하나만을 가지고 결정하는 별거 아닌 알고리즘도 높은 정확도를 나타내는 상황이 발생하는 것.

#### 사이킷런 BaseEstimator클래스 상속받아 아무런 학습하지않고 성별에 따라 생존자를 예측하는 단순한 Classifier를 생성

##### 사이킷런은 BaseEstimator를 상속받으면 Customized형태의 Estimator를 개발자가 생성할수있다.
##### 생성할 MyDummyClassifier클래스는 학습을 수행하는 fit()메서드는 아무런수행하지않으며, 예측을 수행하는 predict()메서드는 단순히  sex피쳐가 1이면0 으로 구분하는 단순한 classifier이다.

In [2]:
import numpy as np
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # fit()메서드는 아무것도 학습하지않음.
    def fit(self, X, y=None):
        pass
    
    # predict()메서드는 단순히 sex피쳐가 1이면0, 그렇지 않으면 1로 예측함.
    def predict(self, X):
        pred = np.zeros((X.shape[0],1))
        for i in range (X.shape[0]) :
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else :
                pred[i] = 1
        
        return pred
    

이제 생성된 커스텀클래스를 이용해 타이타닉 생존자 예측 수행.

In [21]:
# 데이터 가공 내역 정리, 함수로 만들어 쉽게 재사용
# 데이터 전처리를 전체적으로 호출하는 함수 transform_featurs(): null처리, 포매팅, 인코딩 수행 내부 함수로 구성

from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할.
titanic_df = pd.read_csv("titanic_train.csv")
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived',axis=1)
X_titanic_df = transform_features(X_titanic_df)  #전처리함수
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

#위에서 생성한 Dummy Classifier를 이용해 학습/예측/평가 수행
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는 : {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도는 : 0.7877


#### 이렇게 단순한 알고리즘으로 예측을 하더라도 데이터의 구성에 따라 정확도 결과는 약78.77%로  꽤 높은 수치가 나올수있기에 정확도를 평가 지표로 사용할 때는 매우 신중해야 합니다.


#### 특히, 정확도는 불균현항(imbalanced)레이블 값 분포에서 ML모델의 성능을 판단할 경우, 적합한 평가 지표가 아니다.
#### 예를 들어, 100개의 데이터중에 90개 데이터 레이블이 0, 단 10개의 데이터 레이블이 1이라고한다면 무조건 0으로 예측결과를 반환하는 ML모델의 경우라도 정확도가 90%가 된다

#### 유명한 MNIST데이터세트를 변환해 불균형한 데이터 세트로 만든뒤에 정확도 지표 적용시 어떤 문제가 발생하는지 살펴보자
MNIST설명 URL : https://sdc-james.gitbook.io/onebook/4.-and/5.1./5.1.3.-mnist-dataset

MNIST데이터세트는 0부터9까지의 숫자 이미지의 픽셀 정보를 가지고있음.이를 기반으로 숫자Digit를 예측하는데 사용됨.

사이킷런은 load_digits() API를 통해 제공한다.

이것을 레이블값이 7인것만 True, 나머지는 모두 False로 변환하여 이진분류 문제로 변형해보자(즉, 전체 데이터의 10%만 True, 나머지 90%는 False인 불균형한 데이터 세트로 변형하는것)

이렇게 불균형한 데이터 세트에 0으로 예측하는 classifier를 이용해 정확도를 측정하면 약90%에 가까운 예측 정확도를 나타낸다.

**아무것도하지않고 무조건 특정한 결과로 찍어도 데이터 분포도가 균일하지 않은 경우 높은 수치가 나타날수있는 것이 정확도 평가지표의 맹점이다**

In [1]:
#불균형한 데이터 세트와 Dummy Classifier를 생성
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    #입력값으로 들어오는 X데이터세트의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X):
        return np.zeros((len(X),1), dtype=bool)
    
    
#사이킷런의 내장 데이터세트를 이용해 MNIST로딩
digits = load_digits()

#digits번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7번이 아니면 False이고 0으로 변환.
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

#### 다음으로 불균형한 데이터로 생성한 y_test의 데이터분포도를 확인하고 MyFakeClassifier를 이용해 예측과 평가를 수행해보자

In [14]:
#불균형한 레이블 데이터 분포도 확인
print('레이블 테스트 세트 크기 :' , y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())


#dummy classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracy_score(y_test, fakepred)))

레이블 테스트 세트 크기 : (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 0으로 하여도 정확도는:0.900


#### 이처럼 정확도 평가지표는 불균형한 레이블 데이터 세트에서는 성능 수치로 사용돼서는 안됩니다.
### 정확도가 가지는 분류 평가 지표로서 이러한 한계점을 극복하기 위해 여러가지 분류 지표와 함께 적용해야한다. 먼저 True/False, Positive/Negative의 4분면으로 구성되는 오차행렬(Confusion Matrix)에 대해 설명해보자

## 02.오차행렬(Confusion Matrix)

### 이진 분류에서 성능지표로 잘활용됨. 이진분류의 예측 오류가 얼마인지와 더불어 어떠한 유형의 예측 오류가 발생하고 있는지 함께 나타내는 지표

#### 4분면 행렬에서 실제 레이블 클래스 값과 예측 레이블 클래스 값이 어떠한 유형을 가지고 매핑되는지를 나타냄.

<img src="confusionmatrix.jpg">

예측 클래스와 실제 클래스의 Positive결정값과 Negative결정값의 결합에 따라 결정됨.

TN : 예측값을 Negative값0으로 예측했고 실제 값도 Negative 값 0 

FP : 예측값을 Positive 값1로 예측했는데 실제 값은 Negative 값 0

FN : 예측값을 Negative값0으로 예측했는데 실제 값은 Positive값1

TP : 예측값을 Positive값1로 예측했는데 실제 값 역시 Positive값1

사이킷런은 오차행렬을 구하기위헤 confusion_matrix() API를 제공합니다.

정확도 예제에서 다룬 MyFakeClassifier의 예측 성능 지표를 오차 행렬로 표현해보자

(MyFakeClassifier 예측결과인 fakepred와 실제 결과인 y_test를 인자로 입력)

In [15]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, fakepred)


array([[405,   0],
       [ 45,   0]], dtype=int64)

출력된 오차 행렬은 ndarray 형태


일단, 예측을 무조건 negative(false)로 했기에 4분면중 오른쪽은 0건.

모두 false이므로 false로 예측한 TN이 405개, 그중 45개만 true인데 false로 예측함.(FN=45개)

##### TP,TN,FP,TN값은 Classifier성능의 여러 면모를 판단할 수 있는 기반 정보를 제공합니다. 이값을 조합해 Classifier의 성능을 측정할수있는 주요지표인 정확도(Accuracy), 정밀도(Precision), 재현율(Recall)값을 알수있다.

##### 정확도는 예측값과 실제값이 얼마나 동일한가에 대한 비율만으로 결정된다. 즉 , TN과 TP에 좌우된다.
$정확도 = {(TN + TP) \over ( TN + FP + FN + TP)}$ 

#### 일반적으로, 이진분류모델에서는 많은 데이터중에서 중점적으로 찾아야하는 매우 적은 수의 결괏값에 Positive를 설정해 1값을 부여하는 경우가 많음. 예를들어 사기행위예측모델에서는 사기행위가 Positive 양성으로 1, 정상행위가 Negative음성으로0값으로 설정하는 경우가 일반적이다.

##### 불균형한 이진 분류데이터세트에서는 Positive데이터 건수가 매우 작기때문에 데이터에 기반한 ML알고리즘은 positive보다 negative로 예측 정확도가 높아지는 경향이발생한다. TN은 매우커지고, TP는 매우작아짐. 또한 Negative로 예측할때 정확도가 높기 때문에 FN이 매우작고, FP역시 매우작아진다. 

#### 결과적으로 정확도 지표는 비대칭한 데이터세트에서 Positive에 대한 예측 정확도를 판단하지 못한채 Negative에 대한 예측 정확도만으로도 분류의 정확도가 매우 높게 나타나는 수치적인 판단 오류를 일으키게된다.

### 정확도는 분류모델의 성능을 측정할 수 있는 한 가지 요소일 뿐. 불균형한 데이터세트에서 정확도만으로는 모델 신뢰도가 떨어질수있는 사례를 보았으니, 다음으로는 불균형한데이터세트에서 정확도보다 더 선호되는 평가지표인 정밀도와 재현율에 대해 알아보자.

##### [참고]
<img src='recall_precision.png'>

**민감도(sensitivity) = TP/(TP+FN) = 재현율(recall)**

TP + FN : 전체 양성 개수

TP : 양성으로 판정했는데 실제로 양성인 경우


**특이도(specificity)=TN/(TN+FP)**

TN + FP : 전체 음성 개수

TN : 음성으로 판정했는데 실제로 음성인 경우


#민감도와 특이도는 한 쪽이 증가하면 다른 한 쪽이 감소하는 경향을 보일 수 있습니다.

예를 들어 검사 항목을 모두 양성으로 판정하면 민감도는 1이 되지만

특이도는 0이 되며 반대로 모두 음성으로 판정하면 특이도는 1이 되지만 민감도는 0이 됩니다.

궁극적으로는 민감도와 특이도가 둘 다 높게 나오는 방법을 찾아야 합니다.


**정확도(accuracy) = (TP+TN) / (TP+FN+FP+TN)**

전체 중에 양성과 음성을 맞춘 개수


**정밀도(Precision) = (TP/TP+FP)**

양성이라고 판정한 것 중에 실제 양성 개수

## 03.정밀도(Precision)와 재현율(Recall)

정밀도 : TP/TP+FP

재현율(민감도) : TP/(TP+FN)

### 정밀도 : 예측을 Positive로 한 대상 중, 예측과 실제 값이 Positive로 일치한 데이터의 비율 
### 재현율 : 실제 값이 Positive인 대상 중, 예측과 실제 값이 Positive로 일치한 데이터의 비율

 - 참고: 정밀도는 Positive예측 성능을 더욱 정밀하게 측정하기위한 평가지표로 양성예측도 라고도 불린다

 - 재현율 = 민감도(Sensitivity) = TPR(True Positive Rate)

#### 재현율 ~ 암판단모델, 금융사기(보험사기) 등 

1) 실제 Positive양성 데이터를 Negative로 잘못판단하게되면 업무상 큰영향이 발생하는 경우

   ex. 암환자(Positive)를 음성으로 판단할경우 생명위험. opp. 암이아닌데(negative) 양성으로 판단할경우 재검하면됨.

   ex. 사기거래(Positive)를 Negative로 오판단할경우 회사손해.


2) 보통 재현율이 정밀도 보다 상대적으로 중요한 업무가 많지만. 정밀도가 중요한 경우도있음.

ex.스팸메일(positive)을 negative인 일반메일로 분류할경우 문제없음(사용자불편수준), 실제 일반메일(negative)을 positive(스팸메일)로 분류할경우 메일수신이 안되어 업무차질발생.


재현율과 정밀도 모두 TP를 높이는데 초점이맞추어져있으나,
재현율은 FN(실제 P, 예측 N)을 낮추는데
정밀도는 FP를 낮추는데 초점을 맞춤.

모두 높은수치를 얻는것이 좋다. 반면에 둘중 한평가만 지표가 매우높고 다른하나는 매우 낮을 경우 바람직하지않음

앞의 타이타닉에선 정확도에만 초점을 맞추었다면, 이번에 오차행렬 및 정밀도, 재현율을 모두 구하여 예측성능을 평가해보자
사이킷런은 API를 제공한다(정밀도 precision_score(), 재현율 recall_score())

평가의 편의를 위해 confusion matrix, accracy, precision, recall 등 한번에 호출하는 함수를 만들자. 그리고 타이타닉 데이터를 다시 로드하여 가공한 뒤 로지스틱 회귀로 분류를 수행하겠습니다

In [15]:
from sklearn.metrics import(accuracy_score, precision_score, recall_score, confusion_matrix)
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도:{0:4f}, 정밀도:{1:.4f}, 재현율:{2:.4f}'.format(accuracy, precision, recall))

이제 로지스틱 회귀기반으로 타이타닉 생존자를 예측하고 confusion matrix, accuracy, precision, recall 평가를 수행합니다


In [83]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#원본데이터를 재로딩, 데이터 가공, 학습 데이터/테그트 데이터 분할


titanic_df = pd.read_csv('titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived',axis=1)
X_titanic_df = transform_features(X_titanic_df)

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.20, random_state=11)
lr_clf = LogisticRegression(max_iter=1000)

In [84]:
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

오차 행렬
[[104  14]
 [ 13  48]]
정확도:0.849162, 정밀도:0.7742, 재현율:0.7869


### 정밀도/재현율 트레이드오프

분류의 결정 임곗값(Threshold)을 조정하여 정밀도와 재현율 수치를 높일수 있다.

하지만 상호보완적인 평가지표이기 때문에 한쪽을 높이면 다른쪽은 떨어지기쉽다 --> 이를 정밀도/재현율의 Trade-off라고한다

사이킷런의 분류알고리즘은
1) 예측데이터가 특정레이블(Lable,결정클래스값)에 속하는지를 계산하기위해 먼저 개별 레이블별로 결정확률을 구한다

2) 예측확률이 큰 레이블값으로 예측하게 된다.

ex) 0이될확률이 10%, 1이될 확률이 90%로 예측되면, 최종 예측은 더큰 확률을 가진 1로예측한다.

일반적으로, 이진 분류에서는 이 임계값을 **0.5 즉 50%**로 정하고 이보다 크면 Positive,작으면 Negative로 결정한다

predict_proba() : 개별 데이터별로 예측확률을 반환하는 메서드

테스트 피처 데이터세트를 주면 테스트 피처 레코드의 개별 클래스 예측확률을 반환

cf. predict()와 유사. 반환결과가 예측결과 클래스값이 아닌 예측확률결과

반환값 ndarray는 첫 번째 칼럼이 클래스값0에 대한 예측확률,

두번째 칼럼이 클래스 값1에 대한 예측확률

In [86]:
pred_proba = lr_clf.predict_proba(X_test)
pred = lr_clf.predict(X_test)
print('pred_proba()결과 Shape : {0}'.format(pred_proba.shape))
print('pred_proba array에서 앞 3개만 샘플로 추출 \n:', pred_proba[:3])

#예측확률 array와 예측결괏값 array를 병합해 예측 확률과 결괏값을 한눈에 확인
pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1,1)], axis=1)
print('두개의 class중에서 더큰 확률을 클래스 값으로 예측 \n', pred_proba_result[:3])

pred_proba()결과 Shape : (179, 2)
pred_proba array에서 앞 3개만 샘플로 추출 
: [[0.46216576 0.53783424]
 [0.8787286  0.1212714 ]
 [0.87716197 0.12283803]]
두개의 class중에서 더큰 확률을 클래스 값으로 예측 
 [[0.46216576 0.53783424 1.        ]
 [0.8787286  0.1212714  0.        ]
 [0.87716197 0.12283803 0.        ]]


첫번째 칼럼과 두번째 칼럼을 더하면 1이다.(예측확률을 반환함)

두개의 칼럼중에서 더 큰 확률 값으로predict()메서드가 최종 예측하고있음.